In [1]:
from pathlib import Path
import sys
from snowflake.snowpark import Session
import modin.pandas as pd
import snowflake.snowpark.modin.plugin

connection_parameters_path = str(Path("__file__").absolute().parent.parent.parent.parent)
sys.path.append(connection_parameters_path)
from tests.parameters import CONNECTION_PARAMETERS

session = Session.builder.configs(CONNECTION_PARAMETERS).create()


In [2]:
df = pd.read_csv("data/yellow_tripdata_2015-01.csv")

In [3]:
df.isna().sum()

VENDORID                     0
TPEP_PICKUP_DATETIME         0
TPEP_DROPOFF_DATETIME        0
PASSENGER_COUNT              0
TRIP_DISTANCE                0
RATECODEID                   0
STORE_AND_FWD_FLAG           0
PULOCATIONID                 0
DOLOCATIONID                 0
PAYMENT_TYPE                 0
FARE_AMOUNT                  0
EXTRA                        0
MTA_TAX                      0
TIP_AMOUNT                   0
TOLLS_AMOUNT                 0
IMPROVEMENT_SURCHARGE        0
TOTAL_AMOUNT                 0
CONGESTION_SURCHARGE     15000
AIRPORT_FEE              15000
dtype: int64

In [9]:
def chained_one(df):
        return (df
                .drop(columns=['PULOCATIONID','DOLOCATIONID'])
                # dropna with axis==1 not yet supported
                #.dropna(axis='columns')
               )

df_one = chained_one(df)
df_one

NotImplementedError: Snowpark pandas dropna API doesn't yet support axis == 1

In [6]:
df_one['TPEP_PICKUP_DATETIME'] = pd.to_datetime(df_one['TPEP_PICKUP_DATETIME'])
df_one['TPEP_DROPOFF_DATETIME'] = pd.to_datetime(df_one['TPEP_DROPOFF_DATETIME'])
df_one.sort_values(by='TPEP_PICKUP_DATETIME', ascending=True)

,VENDORID,TPEP_PICKUP_DATETIME,TPEP_DROPOFF_DATETIME,PASSENGER_COUNT,TRIP_DISTANCE,RATECODEID,STORE_AND_FWD_FLAG,PAYMENT_TYPE,FARE_AMOUNT,EXTRA,MTA_TAX,TIP_AMOUNT,TOLLS_AMOUNT,IMPROVEMENT_SURCHARGE,TOTAL_AMOUNT,CONGESTION_SURCHARGE,AIRPORT_FEE
10799,2,2015-01-01 00:00:23,2015-01-01 00:01:25,1,0.24,1,False,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,None,None
822,1,2015-01-01 00:01:07,2015-01-01 00:11:34,1,0.80,1,False,2,7.5,0.5,0.5,0.00,0.0,0.0,8.80,None,None
10330,2,2015-01-01 00:02:08,2015-01-01 00:15:19,2,3.65,1,False,1,14.0,0.5,0.5,3.62,0.0,0.3,18.92,None,None
2282,1,2015-01-01 00:02:24,2015-01-01 00:11:05,2,1.10,1,False,1,7.5,0.5,0.5,1.00,0.0,0.0,9.80,None,None
11184,1,2015-01-01 00:02:31,2015-01-01 00:07:06,4,0.90,1,False,1,5.0,0.5,0.5,1.25,0.0,0.0,7.55,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9483,1,2015-01-01 14:59:32,2015-01-01 15:08:07,1,1.50,1,False,2,8.0,0.0,0.5,0.00,0.0,0.0,8.80,None,None
331,1,2015-01-01 14:59:34,2015-01-01 15:06:39,1,1.10,1,False,1,6.5,0.0,0.5,1.50,0.0,0.0,8.80,None,None
10685,1,2015-01-01 14:59:37,2015-01-01 15:09:11,3,2.90,1,False,2,11.0,0.0,0.5,0.00,0.0,0.0,11.80,None,None
12300,2,2015-01-01 14:59:48,2015-01-01 15:09:37,2,3.38,1,False,1,12.0,0.0,0.5,2.00,0.0,0.3,14.80,None,None


In [7]:
def chained_two(df):
        return (df
                .sort_values(by='TPEP_PICKUP_DATETIME', ascending=True)
                )

df_two = chained_two(df_one)
df_two

,VENDORID,TPEP_PICKUP_DATETIME,TPEP_DROPOFF_DATETIME,PASSENGER_COUNT,TRIP_DISTANCE,RATECODEID,STORE_AND_FWD_FLAG,PAYMENT_TYPE,FARE_AMOUNT,EXTRA,MTA_TAX,TIP_AMOUNT,TOLLS_AMOUNT,IMPROVEMENT_SURCHARGE,TOTAL_AMOUNT,CONGESTION_SURCHARGE,AIRPORT_FEE
10799,2,2015-01-01 00:00:23,2015-01-01 00:01:25,1,0.24,1,False,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,None,None
822,1,2015-01-01 00:01:07,2015-01-01 00:11:34,1,0.80,1,False,2,7.5,0.5,0.5,0.00,0.0,0.0,8.80,None,None
10330,2,2015-01-01 00:02:08,2015-01-01 00:15:19,2,3.65,1,False,1,14.0,0.5,0.5,3.62,0.0,0.3,18.92,None,None
2282,1,2015-01-01 00:02:24,2015-01-01 00:11:05,2,1.10,1,False,1,7.5,0.5,0.5,1.00,0.0,0.0,9.80,None,None
11184,1,2015-01-01 00:02:31,2015-01-01 00:07:06,4,0.90,1,False,1,5.0,0.5,0.5,1.25,0.0,0.0,7.55,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9483,1,2015-01-01 14:59:32,2015-01-01 15:08:07,1,1.50,1,False,2,8.0,0.0,0.5,0.00,0.0,0.0,8.80,None,None
331,1,2015-01-01 14:59:34,2015-01-01 15:06:39,1,1.10,1,False,1,6.5,0.0,0.5,1.50,0.0,0.0,8.80,None,None
10685,1,2015-01-01 14:59:37,2015-01-01 15:09:11,3,2.90,1,False,2,11.0,0.0,0.5,0.00,0.0,0.0,11.80,None,None
12300,2,2015-01-01 14:59:48,2015-01-01 15:09:37,2,3.38,1,False,1,12.0,0.0,0.5,2.00,0.0,0.3,14.80,None,None


In [8]:
def chained_three(df):
        return (df
                .loc[:, ['PASSENGER_COUNT','TRIP_DISTANCE', 'FARE_AMOUNT', 'EXTRA', 'MTA_TAX', 'TIP_AMOUNT', 'TOLLS_AMOUNT', 'TOTAL_AMOUNT']]
                .groupby(by=["PASSENGER_COUNT"]).mean()
                .reset_index()
                # TODO: SNOW-800922 should support df.query
                #.query('MTA_TAX >= TOLLS_AMOUNT')
                )

df_three = chained_three(df_two)
df_three

,PASSENGER_COUNT,TRIP_DISTANCE,FARE_AMOUNT,EXTRA,MTA_TAX,TIP_AMOUNT,TOLLS_AMOUNT,TOTAL_AMOUNT
0,0,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.300000
1,1,3.289381,12.882922,0.265203,0.496297,1.344067,0.186904,15.473178
2,2,3.466865,13.582750,0.287996,0.493576,1.434420,0.334787,16.434739
3,3,3.742341,13.974659,0.303409,0.492614,1.190011,0.370841,16.628807
4,4,3.419715,13.781524,0.291667,0.489837,1.200122,0.352337,16.412439
5,5,3.572310,13.533213,0.283995,0.496390,1.295716,0.286715,16.189892
6,6,3.228722,12.527835,0.256701,0.500000,1.462722,0.186825,15.231608


In [ ]:
# import pandas as pandas
# df_true = pandas.read_csv('true.csv')
# df_three.to_csv('output.csv')
# df_output = pandas.read_csv('output.csv')

In [ ]:
from pandas.testing import assert_frame_equal
# assert_frame_equal(df_true, df_output)

In [ ]:
#!rm output.csv